In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import yfinance as yf
import os
from dotenv import load_dotenv
from scipy.signal import correlate as cr
#adf test
from statsmodels.tsa.stattools import adfuller
#cross correlation
from scipy import signal

#derivate
from sympy import diff, Symbol

Importando dados

In [2]:
#importando dados - INDICADORES
load_dotenv()

path_dados_trimestrais = os.getenv("PATH_DADOS_TRIMESTRAIS")
path_dados_anuais = os.getenv("PATH_DADOS_ANUAIS")

dados_trimestrais = pd.read_csv(path_dados_trimestrais + "base_dados_tri.csv", sep=";")
dados_anuais = pd.read_csv(path_dados_anuais + "base_dados_anual.csv", sep=";")

In [3]:
dados_trimestrais.info()
dados_anuais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1833179 entries, 0 to 1833178
Data columns (total 16 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Unnamed: 0     int64  
 1   CNPJ_CIA       object 
 2   DT_REFER       object 
 3   VERSAO         int64  
 4   DENOM_CIA      object 
 5   CD_CVM         int64  
 6   GRUPO_DFP      object 
 7   MOEDA          object 
 8   ESCALA_MOEDA   object 
 9   ORDEM_EXERC    object 
 10  DT_INI_EXERC   object 
 11  DT_FIM_EXERC   object 
 12  CD_CONTA       object 
 13  DS_CONTA       object 
 14  VL_CONTA       float64
 15  ST_CONTA_FIXA  object 
dtypes: float64(1), int64(3), object(12)
memory usage: 223.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418588 entries, 0 to 418587
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     418588 non-null  int64  
 1   CNPJ_CIA       418588 non-null  object 
 2   DT_REFER       418588 

Removendo coluna não utilizada

In [4]:
dados_trimestrais.drop(columns=["Unnamed: 0"], inplace=True)
dados_anuais.drop(columns=["Unnamed: 0"], inplace=True)


In [5]:
print("Valores nulos:")
print(dados_trimestrais.isnull().sum())
print("\nValores Únicos: ")
for col in dados_trimestrais.columns:
    print(f"{col}: {dados_trimestrais[col].nunique()}")
dados_trimestrais.describe()
print("\nValores duplicados: ")
duplicados = dados_trimestrais.duplicated()
print(f"Total de duplicados: {duplicados.sum()}")
print(f"Tamanho inicial: {dados_trimestrais.shape}")
dados_trimestrais.drop_duplicates(inplace=True)
print(f"Tamanho final: {dados_trimestrais.shape}")



Valores nulos:
CNPJ_CIA         0
DT_REFER         0
VERSAO           0
DENOM_CIA        0
CD_CVM           0
GRUPO_DFP        0
MOEDA            0
ESCALA_MOEDA     0
ORDEM_EXERC      0
DT_INI_EXERC     0
DT_FIM_EXERC     0
CD_CONTA         0
DS_CONTA         0
VL_CONTA         0
ST_CONTA_FIXA    0
dtype: int64

Valores Únicos: 
CNPJ_CIA: 710
DT_REFER: 81
VERSAO: 7
DENOM_CIA: 732
CD_CVM: 714
GRUPO_DFP: 1
MOEDA: 1
ESCALA_MOEDA: 2
ORDEM_EXERC: 2
DT_INI_EXERC: 125
DT_FIM_EXERC: 89
CD_CONTA: 269
DS_CONTA: 4933
VL_CONTA: 355946
ST_CONTA_FIXA: 2

Valores duplicados: 
Total de duplicados: 2934
Tamanho inicial: (1833179, 15)
Tamanho final: (1830245, 15)


In [6]:
print("Valores nulos:")
print(dados_anuais.isnull().sum())
print("\nValores Únicos ")
for col in dados_anuais.columns:
    print(f"{col}: {dados_anuais[col].nunique()}")
dados_anuais.describe()
print("\nValores duplicados: ")
duplicados = dados_anuais.duplicated()
print(f"Total de duplicados: {duplicados.sum()}")
print(f"Tamanho inicial: {dados_anuais.shape}")
dados_anuais.drop_duplicates(inplace=True)
print(f"Tamanho final: {dados_anuais.shape}")

Valores nulos:
CNPJ_CIA         0
DT_REFER         0
VERSAO           0
DENOM_CIA        0
CD_CVM           0
GRUPO_DFP        0
MOEDA            0
ESCALA_MOEDA     0
ORDEM_EXERC      0
DT_INI_EXERC     0
DT_FIM_EXERC     0
CD_CONTA         0
DS_CONTA         0
VL_CONTA         0
ST_CONTA_FIXA    0
dtype: int64

Valores Únicos 
CNPJ_CIA: 725
DT_REFER: 53
VERSAO: 8
DENOM_CIA: 750
CD_CVM: 730
GRUPO_DFP: 1
MOEDA: 1
ESCALA_MOEDA: 2
ORDEM_EXERC: 2
DT_INI_EXERC: 107
DT_FIM_EXERC: 60
CD_CONTA: 252
DS_CONTA: 3555
VL_CONTA: 128663
ST_CONTA_FIXA: 2

Valores duplicados: 
Total de duplicados: 1375
Tamanho inicial: (418588, 15)
Tamanho final: (417213, 15)


Removendo colunas que não agregam valor ao tema. GRUPO_DFP e MOEDA possuem valor único e o número de versão do documento entregue não contribui para a análise.

In [7]:
dados_trimestrais.drop(columns=["GRUPO_DFP", "VERSAO", "MOEDA"], inplace=True)
dados_anuais.drop(columns=["GRUPO_DFP", "VERSAO", "MOEDA"], inplace=True)

Convertendo as colunas de data para o tipo datetime

In [8]:
dados_trimestrais['DT_REFER'] = pd.to_datetime(dados_trimestrais['DT_REFER'])
dados_trimestrais['DT_INI_EXERC'] = pd.to_datetime(dados_trimestrais['DT_INI_EXERC'],format='ISO8601')
dados_trimestrais['DT_FIM_EXERC'] = pd.to_datetime(dados_trimestrais['DT_FIM_EXERC'],format='ISO8601')

dados_anuais['DT_REFER'] = pd.to_datetime(dados_anuais['DT_REFER'])
dados_anuais['DT_INI_EXERC'] = pd.to_datetime(dados_anuais['DT_INI_EXERC'],format='ISO8601')
dados_anuais['DT_FIM_EXERC'] = pd.to_datetime(dados_anuais['DT_FIM_EXERC'],format='ISO8601')

Remover a coluna de ESCALA_MOEDA aplicando a escala aos valores da coluna VL_CONTA

In [9]:
print(dados_trimestrais['ESCALA_MOEDA'].unique())
print(dados_trimestrais[['ESCALA_MOEDA', 'VL_CONTA']].head(50))

dados_trimestrais.loc[dados_trimestrais['ESCALA_MOEDA'] == 'MIL', 'VL_CONTA'] *= 1000
print(dados_trimestrais[['ESCALA_MOEDA', 'VL_CONTA']].head(50))

dados_anuais.loc[dados_anuais['ESCALA_MOEDA'] == 'MIL', 'VL_CONTA'] *= 1000
print(dados_anuais[['ESCALA_MOEDA', 'VL_CONTA']].head(50))

dados_trimestrais.drop(columns=["ESCALA_MOEDA"], inplace=True)
dados_anuais.drop(columns=["ESCALA_MOEDA"], inplace=True)



['MIL' 'UNIDADE']
   ESCALA_MOEDA   VL_CONTA
0           MIL  6159172.0
1           MIL  8551982.0
2           MIL  -960060.0
3           MIL -1127021.0
4           MIL  -340660.0
5           MIL  -398353.0
6           MIL  -224418.0
7           MIL  -252475.0
8           MIL  -394982.0
9           MIL  -476193.0
10          MIL  5199112.0
11          MIL  7424961.0
12          MIL -4861068.0
13          MIL -5281346.0
14          MIL        0.0
15          MIL        0.0
16          MIL -5285273.0
17          MIL -5745196.0
18          MIL -1545413.0
19          MIL -1712648.0
20          MIL  -875581.0
21          MIL  -932076.0
22          MIL   -83370.0
23          MIL  -138683.0
24          MIL  -385295.0
25          MIL  -340757.0
26          MIL  -262277.0
27          MIL  -296130.0
28          MIL  -535664.0
29          MIL -1176673.0
30          MIL  -391968.0
31          MIL  -425125.0
32          MIL  -759045.0
33          MIL  -318071.0
34          MIL  -301408.0
35        

Nota-se que a coluna ORDEM_EXEC indica se é a linha é o resultado atual ("ÚLTIMO") ou se é resultado referente ao mesmo período do ano anterior ("PENÚLTIMO").  

In [10]:
dados_trimestrais.query("CD_CVM == 2437 and DS_CONTA == 'Receita de Venda de Bens e/ou Serviços' and DT_FIM_EXERC == '2011-03-31'")

C:\Users\giova\AppData\Local\Temp\ipykernel_189112\830947284.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  dados_trimestrais.query("CD_CVM == 2437 and DS_CONTA == 'Receita de Venda de Bens e/ou Serviços' and DT_FIM_EXERC == '2011-03-31'")


,CNPJ_CIA,DT_REFER,DENOM_CIA,CD_CVM,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
1,00.001.180/0001-26,2011-03-31,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,ÚLTIMO,2011-01-01,2011-03-31,3.01,Receita de Venda de Bens e/ou Serviços,8.551982e+09,S
123576,00.001.180/0001-26,2012-03-31,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,PENÚLTIMO,2011-01-01,2011-03-31,3.01,Receita de Venda de Bens e/ou Serviços,8.551982e+09,S


Neste caso, como os relatórios disponíveis na CVM começam em 2011, é possível resgatar os dados de 2010 a partir destes valores destacados como "PENÚLTIMO". Os demais dados com essa label serão descartados para evitar redundâncias na base.

In [11]:
dados_trimestrais_clean = dados_trimestrais.copy()
dados_trimestrais_clean = dados_trimestrais_clean.query("ORDEM_EXERC == 'ÚLTIMO' or (ORDEM_EXERC == 'PENÚLTIMO' and DT_REFER.dt.year == 2011)").sort_values(by=['DT_FIM_EXERC'], ascending=True)
dados_trimestrais_clean.query("ORDEM_EXERC == 'PENÚLTIMO' and CD_CONTA == '3.01' and CD_CVM == 18970")

dados_anuais = dados_anuais.query("ORDEM_EXERC == 'ÚLTIMO' or (ORDEM_EXERC == 'PENÚLTIMO' and DT_REFER.dt.year == 2011)").sort_values(by=['DT_FIM_EXERC'], ascending=True)
dados_anuais.query("ORDEM_EXERC == 'PENÚLTIMO' and CD_CONTA == '3.01' and CD_CVM == 18970")

dados_trimestrais = dados_trimestrais_clean.copy()

Analisando a coluna CD_CONTA, ela indica o label de qual indicador será expresso o valor na coluna VL_CONTA. Porém, os indicadores possuem grupos hierárquicos, por exemplo, o 3.01 é a Receita de Venda de Bens e/ou Serviços (dependedo do setor de atuação da empresa, pode ser outro tipo de receita como Receitas de Intermediação Financeira, porém de qualquer forma indica a receita da operacional da empresa), neste caso os indicadores de valores 3.01.X indicam que a somatórios deles irá resultados no valor de 3.01. Para realizar a análise deste estudo, serão utilizadas as categorias inicias denotadas por 3.X, as subcategorias 3.X.X não serão utilizadas por não aparecem constantemente todos os meses ou por representar valor pouco simbólicos e redundantes. 

In [12]:
#Utilizando a coluna 'CD_CONTA' para filtrar os indicadores
regex = r"^\d{1}\.\d{2}(?<!99)$"

#filtrar linhas da base_dados_tri que contenham o formato regex no tipo de indicador
print(f'Dados trimestrais antes: {dados_trimestrais.shape}')
print(f'Dados anuais antes: {dados_anuais.shape}')
dados_anuais = dados_anuais[dados_anuais['CD_CONTA'].str.contains(regex)]
dados_trimestrais = dados_trimestrais[dados_trimestrais['CD_CONTA'].str.contains(regex)]
print(f'Dados trimestrais depois: {dados_trimestrais.shape}')
print(f'Dados anuais depois: {dados_anuais.shape}')
dados_trimestrais.info()
dados_anuais.info()

Dados trimestrais antes: (977300, 11)
Dados anuais antes: (222088, 11)
Dados trimestrais depois: (304631, 11)
Dados anuais depois: (68166, 11)
<class 'pandas.core.frame.DataFrame'>
Index: 304631 entries, 0 to 1714528
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   CNPJ_CIA       304631 non-null  object        
 1   DT_REFER       304631 non-null  datetime64[ns]
 2   DENOM_CIA      304631 non-null  object        
 3   CD_CVM         304631 non-null  int64         
 4   ORDEM_EXERC    304631 non-null  object        
 5   DT_INI_EXERC   304631 non-null  datetime64[ns]
 6   DT_FIM_EXERC   304631 non-null  datetime64[ns]
 7   CD_CONTA       304631 non-null  object        
 8   DS_CONTA       304631 non-null  object        
 9   VL_CONTA       304631 non-null  float64       
 10  ST_CONTA_FIXA  304631 non-null  object        
dtypes: datetime64[ns](3), float64(1), int64(1), object(6)
memory usage: 2

Nota-se que há períodos semestrais (apenas primeiro semestre) registrados, para melhor uso as linhas semestrais devem ser removidas. Assim como o quarto trimestre estava ausente e será calculado a partir dos 3 trimestres anteriores e do valor anual obtido na planilha DFP.

In [13]:
def compilado_trimestral(dados_trimestrais: pd.DataFrame, dados_anuais: pd.DataFrame):
    # Filtrar apenas os dados trimestrais válidos
    dados_3tri = dados_trimestrais[
        ((dados_trimestrais['DT_FIM_EXERC'] - dados_trimestrais['DT_INI_EXERC']) < datetime.timedelta(days=100)) &
        ((dados_trimestrais['DT_FIM_EXERC'] - dados_trimestrais['DT_INI_EXERC']) > datetime.timedelta(days=31))
    ]

    # Agrupar os dados trimestrais por ano e conta, somando os valores
    dados_3tri['ANO'] = dados_3tri['DT_FIM_EXERC'].dt.year
    soma_3tri = dados_3tri.groupby(['CD_CVM', 'CD_CONTA', 'ANO'], as_index=False).agg({
        'VL_CONTA': 'sum',
        'DS_CONTA': 'first',
        'ST_CONTA_FIXA': 'first',
        'CNPJ_CIA': 'first',
        'DENOM_CIA': 'first',
        'DT_FIM_EXERC': 'first'
    })

    # Adicionar os dados do 4º trimestre
    new_data = []
    for ano in range(2010, 2024):
        # Filtrar os dados anuais para o ano atual
        dados_anuais_ano = dados_anuais[dados_anuais['DT_FIM_EXERC'].dt.year == ano]

        # Combinar os dados trimestrais e anuais
        for _, row in soma_3tri[soma_3tri['ANO'] == ano].iterrows():
            cd_conta = row['CD_CONTA']
            total_3tri = row['VL_CONTA']
            cd_cvm = row['CD_CVM']

            # Verificar se existe o dado anual correspondente
            total_anual = dados_anuais_ano[(dados_anuais_ano['CD_CONTA'] == cd_conta) & (dados_anuais_ano['CD_CVM'] == cd_cvm) ]['VL_CONTA']
            if not total_anual.empty:
                total_anual = total_anual.iloc[0]
                value_4t = total_anual - total_3tri
            else:
                print(f"Vazio {cd_conta} - {cd_cvm} - {ano}")
                value_4t = 0.0
                # Criar o registro do 4º trimestre
            new_data.append({
                'CD_CVM': row['CD_CVM'],
                'CD_CONTA': cd_conta,
                'DT_INI_EXERC': datetime.datetime(ano, 10, 1),
                'DT_FIM_EXERC': datetime.datetime(ano, 12, 31),
                'DS_CONTA': row['DS_CONTA'],
                'VL_CONTA': value_4t,
                'ST_CONTA_FIXA': row['ST_CONTA_FIXA'],
                'CNPJ_CIA': row['CNPJ_CIA'],
                'DT_REFER': datetime.datetime(ano, 12, 31),
                'DENOM_CIA': row['DENOM_CIA'],
                'ORDEM_EXERC': 'ÚLTIMO'
            })

    # Criar um DataFrame com os novos dados
    new_data_df = pd.DataFrame(new_data)

    # Concatenar os dados trimestrais e os novos dados
    dados_trimestrais_completos = pd.concat([dados_3tri, new_data_df], ignore_index=True)

    dados_trimestrais_completos.drop(columns=["ANO"], inplace=True)

    return dados_trimestrais_completos

In [14]:
dados_trimestrais_tratados = compilado_trimestral(dados_trimestrais, dados_anuais)

C:\Users\giova\AppData\Local\Temp\ipykernel_189112\3617506356.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_3tri['ANO'] = dados_3tri['DT_FIM_EXERC'].dt.year


Vazio 3.01 - 21512 - 2010
Vazio 3.02 - 21512 - 2010
Vazio 3.03 - 21512 - 2010
Vazio 3.04 - 21512 - 2010
Vazio 3.05 - 21512 - 2010
Vazio 3.06 - 21512 - 2010
Vazio 3.07 - 21512 - 2010
Vazio 3.08 - 21512 - 2010
Vazio 3.09 - 21512 - 2010
Vazio 3.01 - 21660 - 2010
Vazio 3.02 - 21660 - 2010
Vazio 3.03 - 21660 - 2010
Vazio 3.04 - 21660 - 2010
Vazio 3.05 - 21660 - 2010
Vazio 3.06 - 21660 - 2010
Vazio 3.07 - 21660 - 2010
Vazio 3.08 - 21660 - 2010
Vazio 3.09 - 21660 - 2010
Vazio 3.10 - 21660 - 2010
Vazio 3.11 - 21660 - 2010
Vazio 3.01 - 21709 - 2010
Vazio 3.02 - 21709 - 2010
Vazio 3.03 - 21709 - 2010
Vazio 3.04 - 21709 - 2010
Vazio 3.05 - 21709 - 2010
Vazio 3.06 - 21709 - 2010
Vazio 3.07 - 21709 - 2010
Vazio 3.08 - 21709 - 2010
Vazio 3.09 - 21709 - 2010
Vazio 3.10 - 21709 - 2010
Vazio 3.11 - 21709 - 2010
Vazio 3.01 - 22012 - 2010
Vazio 3.02 - 22012 - 2010
Vazio 3.03 - 22012 - 2010
Vazio 3.04 - 22012 - 2010
Vazio 3.05 - 22012 - 2010
Vazio 3.06 - 22012 - 2010
Vazio 3.07 - 22012 - 2010
Vazio 3.08 -

Removendo a coluna de DT_INICIO_EXERC, DT_REFER, ST_CONTA_FIXA, CNPJ_CIA, DENOM_CIA e ORDEM_EXERC. E ordenando pela coluna DT_FIM_EXERC

In [15]:
dados_trimestrais_tratados.drop(columns=["DT_INI_EXERC", "DT_REFER", "ORDEM_EXERC", "ST_CONTA_FIXA", "CNPJ_CIA"], inplace=True)
dados_trimestrais_tratados.sort_values(by=['DT_FIM_EXERC'], ascending=True, inplace=True)

In [16]:
dados_trimestrais_tratados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239206 entries, 0 to 182206
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   DENOM_CIA     239206 non-null  object        
 1   CD_CVM        239206 non-null  int64         
 2   DT_FIM_EXERC  239206 non-null  datetime64[ns]
 3   CD_CONTA      239206 non-null  object        
 4   DS_CONTA      239206 non-null  object        
 5   VL_CONTA      239206 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 12.8+ MB


In [17]:
dados_trimestrais_tratados['CD_CONTA'].unique()

array(['3.01', '3.04', '3.05', '3.06', '3.02', '3.03', '3.11', '3.07',
       '3.10', '3.08', '3.09', '3.13', '3.12'], dtype=object)

Montando uma tabela de de/para de códigos e descrição dos indicadores

In [18]:
sorted(
    list(
        dados_trimestrais_tratados[['DS_CONTA', 'CD_CONTA']]
        .drop_duplicates()
        .itertuples(index=False, name=None)
    ),
    key=lambda x: x[1]
)

[('Receita de Venda de Bens e/ou Serviços', '3.01'),
 ('Receitas da Intermediação Financeira', '3.01'),
 ('Receitas das Operações', '3.01'),
 ('0', '3.01'),
 ('Receitas de Intermediação Financeira', '3.01'),
 ('Receitas das Atividades Seguradoras/Resseguradoras', '3.01'),
 ('Custo dos Bens e/ou Serviços Vendidos', '3.02'),
 ('Despesas da Intermediação Financeira', '3.02'),
 ('Sinistros e Despesas das Operações', '3.02'),
 ('Despesas de Intermediação Financeira', '3.02'),
 ('Despesas da Atividade Seguradora/Resseguradora', '3.02'),
 ('Resultado Bruto', '3.03'),
 ('Resultado Bruto Intermediação Financeira', '3.03'),
 ('Resultado Bruto de Intermediação Financeira', '3.03'),
 ('Despesas/Receitas Operacionais', '3.04'),
 ('Despesas Administrativas', '3.04'),
 ('Outras Despesas/Receitas Operacionais', '3.04'),
 ('Outras Despesas e Receitas Operacionais', '3.04'),
 ('Resultado Antes do Resultado Financeiro e dos Tributos', '3.05'),
 ('Outras Receitas e Despesas Operacionais', '3.05'),
 ('Resu

Pivotando a tabela para que os códigos de indicadores indicadores sejam as colunas

In [19]:
# Transformar os valores de 'CD_CONTA' em colunas e usar 'VL_CONTA' como conteúdo
dados_pivot = dados_trimestrais_tratados.pivot_table(
    index=['CD_CVM', 'DENOM_CIA', 'DT_FIM_EXERC'],  # Colunas que permanecerão como índice
    columns='CD_CONTA',  # Coluna que será transformada em colunas
    values='VL_CONTA',  # Valores que preencherão as novas colunas
    aggfunc='sum'  # Função de agregação (caso haja duplicatas)
).reset_index()  # Resetar o índice para transformar em DataFrame normal

In [20]:
dados_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21662 entries, 0 to 21661
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CD_CVM        21662 non-null  int64         
 1   DENOM_CIA     21662 non-null  object        
 2   DT_FIM_EXERC  21662 non-null  datetime64[ns]
 3   3.01          21661 non-null  float64       
 4   3.02          21661 non-null  float64       
 5   3.03          21662 non-null  float64       
 6   3.04          21662 non-null  float64       
 7   3.05          21662 non-null  float64       
 8   3.06          21661 non-null  float64       
 9   3.07          21662 non-null  float64       
 10  3.08          21662 non-null  float64       
 11  3.09          21662 non-null  float64       
 12  3.10          21314 non-null  float64       
 13  3.11          21314 non-null  float64       
 14  3.12          137 non-null    float64       
 15  3.13          137 non-null    float6

Verificando que em alguns momentos os nomes das companhias mudam, porém seus códigos cd_cvm são os mesmos. Em casos de recuperação judicial é adicionado o termo "Em recuperação judicial" no nome da companhia a partir do período que ela iniciou com o processo.

In [21]:
codigo_e_descricoes = list(dados_pivot[['CD_CVM', 'DENOM_CIA']].drop_duplicates().itertuples(index=False, name=None))

In [22]:
codigos = [x[0] for x in codigo_e_descricoes]
from collections import Counter
contagem = Counter(codigos)
repetidos = {k: v for k, v in contagem.items() if v > 1}
print(repetidos)

{701: 2, 3395: 2, 4081: 2, 5762: 2, 8575: 2, 12190: 2, 12823: 2, 14109: 2, 15423: 2, 18376: 2, 18821: 2, 19100: 2, 19550: 2, 20010: 2, 21636: 2, 25194: 2, 25224: 2, 25461: 2}


In [23]:
dados_pivot[dados_pivot['CD_CVM'] == 4081]

CD_CONTA,CD_CVM,DENOM_CIA,DT_FIM_EXERC,3.01,3.02,3.03,3.04,3.05,3.06,3.07,3.08,3.09,3.10,3.11,3.12,3.13
1506,4081,CIA TECIDOS SANTANENSE,2010-03-31,70347000.0,-52191000.0,18156000.0,-10000000.0,8156000.0,-1373000.0,6783000.0,-2042000.0,4741000.0,0.0,4741000.0,NaN,NaN
1507,4081,CIA TECIDOS SANTANENSE,2010-06-30,75924000.0,-58107000.0,17817000.0,-10802000.0,7015000.0,-1708000.0,5307000.0,-1519000.0,3788000.0,0.0,3788000.0,NaN,NaN
1508,4081,CIA TECIDOS SANTANENSE,2010-09-30,83132000.0,-64472000.0,18660000.0,-11507000.0,7153000.0,-1416000.0,5737000.0,-1856000.0,3881000.0,0.0,3881000.0,NaN,NaN
1509,4081,CIA TECIDOS SANTANENSE,2010-12-31,86941000.0,-60113000.0,26828000.0,-12629000.0,14199000.0,-1857000.0,12342000.0,-2028000.0,10314000.0,0.0,10314000.0,NaN,NaN
1510,4081,CIA TECIDOS SANTANENSE,2011-03-31,96523000.0,-69025000.0,27498000.0,-11768000.0,15730000.0,-1369000.0,14361000.0,-3856000.0,10505000.0,0.0,10505000.0,NaN,NaN
1511,4081,CIA TECIDOS SANTANENSE,2011-06-30,94050000.0,-77125000.0,16925000.0,-11351000.0,5574000.0,-3096000.0,2478000.0,-599000.0,1879000.0,0.0,1879000.0,NaN,NaN
1512,4081,CIA TECIDOS SANTANENSE,2011-09-30,93456000.0,-72141000.0,21315000.0,-12477000.0,8838000.0,-439000.0,8399000.0,-783000.0,7616000.0,0.0,7616000.0,NaN,NaN
1513,4081,CIA TECIDOS SANTANENSE,2011-12-31,86346000.0,-67179000.0,19167000.0,-14714000.0,4453000.0,-1574000.0,2879000.0,1493000.0,4372000.0,0.0,4372000.0,NaN,NaN
1514,4081,CIA TECIDOS SANTANENSE,2012-03-31,93298000.0,-70844000.0,22454000.0,-5853000.0,16601000.0,-2517000.0,14084000.0,-4320000.0,9764000.0,0.0,9764000.0,NaN,NaN
1515,4081,CIA TECIDOS SANTANENSE,2012-06-30,94799000.0,-69919000.0,24880000.0,-13157000.0,11723000.0,-587000.0,11136000.0,-3197000.0,7939000.0,0.0,7939000.0,NaN,NaN


Salvar planilha de dados tratados

In [24]:
path_dados_tratados = os.getenv("PATH_DADOS_TRATADOS")
dados_pivot.to_csv(path_dados_tratados + "dados_indicadores_tratados.csv", index=False)